In [1]:
!pip install -U pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
from pypdf import PdfReader
from google.colab import userdata
import re
import nltk
nltk.download("punkt")
nltk.download('punkt_tab')
import requests
import json
import numpy as np
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [65]:
reader = PdfReader('/content/Buku-Panduan-_-Penggunaan-Generative-AI-pada-Pembelajaran-di-Perguruan-Tinggi-cetak.pdf')

knowledge_base = ""

for idx, page in enumerate(reader.pages):
    if 51 <= idx <= 80:   # extract pages 52–81
        knowledge_base += page.extract_text()




In [66]:
print(knowledge_base)

  
Panduan Penggunaan GenAI pada Pembelajaran di Perguruan Tinggi 
 41 
Pemanfaatan teknologi dalam pengerjaan tugas akademik tidak lagi dapat dihindari. 
Dosen perlu mengedepankan proses dan bukan hanya mengandalkan output belajar, 
untuk memotivasi mahasiswa, menimbulkan rasa ingin tahu, serta mendorong cara 
berpikir kritis dan etis melalui umpan balik/kritik yang bersifat evaluatif, reflektif, dan 
kesempatan untuk melakukan revisi.  Metode pembelajaran seperti Students Centered 
Learning (SCL), Higher Order Thinking, dan Flip Learning menjadi alternatif dari kegiatan 
pembelajaran kelas dibanding perkuliahan tradisional. Setiap pertemuan digunakan 
untuk melakukan diskusi yang merefleksikan pemahaman dan penguasaan individu 
mahasiswa terhadap tugas yang mereka kerjakan, misalnya melalui presentasi yang 
diikuti dengan tanya-jawab/diskusi. 
Berbagai bentuk umpan balik dan pertanyaan yang dapat diberikan, antara lain: 
● Meminta mahasiswa menggambarkan cara mereka memanfaatkan GenA

In [75]:
def clean_text(text):
    patterns = [
        r"^DIREKTORAT[^\n]*",
        r"^(KEMENTERIAN|DIREKTORAT JENDERAL)[^\n]*",
        r"Panduan Penggunaan GenAI[^\n]*",
        r"^\d{1,4}$",
        r"Lampiran\s*\d+(\.\d+)*[^\n]*",
        r"https?://[^\s]+",
        r"(^BAB\s+[IVXLC]+\s*\n.*?\n)",

    ]

    for p in patterns:
        text = re.sub(p, "", text, flags=re.IGNORECASE | re.MULTILINE)

    # Remove newline completely (recommended for chunking)
    text = re.sub(r"\n+", " ", text)

    return text.strip()


In [76]:
new_text = clean_text(knowledge_base)
print(new_text)

41  Pemanfaatan teknologi dalam pengerjaan tugas akademik tidak lagi dapat dihindari.  Dosen perlu mengedepankan proses dan bukan hanya mengandalkan output belajar,  untuk memotivasi mahasiswa, menimbulkan rasa ingin tahu, serta mendorong cara  berpikir kritis dan etis melalui umpan balik/kritik yang bersifat evaluatif, reflektif, dan  kesempatan untuk melakukan revisi.  Metode pembelajaran seperti Students Centered  Learning (SCL), Higher Order Thinking, dan Flip Learning menjadi alternatif dari kegiatan  pembelajaran kelas dibanding perkuliahan tradisional. Setiap pertemuan digunakan  untuk melakukan diskusi yang merefleksikan pemahaman dan penguasaan individu  mahasiswa terhadap tugas yang mereka kerjakan, misalnya melalui presentasi yang  diikuti dengan tanya-jawab/diskusi.  Berbagai bentuk umpan balik dan pertanyaan yang dapat diberikan, antara lain:  ● Meminta mahasiswa menggambarkan cara mereka memanfaatkan GenAI untuk  menyelesaikan tugas dan merefleksikan proses yang mereka al

In [77]:
def sentence_chunk_with_overlap(text, max_chars=500, overlap=100):
    sentences = sent_tokenize(text)
    chunks = []
    current = ""

    for s in sentences:
        if len(current) + len(s) + 1 > max_chars:
            chunks.append(current.strip())

            # ambil overlap dari belakang
            current = current[-overlap:] if overlap else ""
            current = current.strip()

        current += " " + s

    if current:
        chunks.append(current.strip())

    return chunks


In [78]:
chunks = sentence_chunk_with_overlap(new_text)

In [80]:
print(chunks)

['41  Pemanfaatan teknologi dalam pengerjaan tugas akademik tidak lagi dapat dihindari. Dosen perlu mengedepankan proses dan bukan hanya mengandalkan output belajar,  untuk memotivasi mahasiswa, menimbulkan rasa ingin tahu, serta mendorong cara  berpikir kritis dan etis melalui umpan balik/kritik yang bersifat evaluatif, reflektif, dan  kesempatan untuk melakukan revisi.', 'lalui umpan balik/kritik yang bersifat evaluatif, reflektif, dan  kesempatan untuk melakukan revisi. Metode pembelajaran seperti Students Centered  Learning (SCL), Higher Order Thinking, dan Flip Learning menjadi alternatif dari kegiatan  pembelajaran kelas dibanding perkuliahan tradisional.', 'lip Learning menjadi alternatif dari kegiatan  pembelajaran kelas dibanding perkuliahan tradisional. Setiap pertemuan digunakan  untuk melakukan diskusi yang merefleksikan pemahaman dan penguasaan individu  mahasiswa terhadap tugas yang mereka kerjakan, misalnya melalui presentasi yang  diikuti dengan tanya-jawab/diskusi.', '

In [89]:
def summarize_chunk(chunk):

    response = requests.post(
        "https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {userdata.get('LLM_API_KEY')}",
            "Content-Type": "application/json",
        },
        json={
            "model": "x-ai/grok-4.1-fast:free",
            "messages": [
                {
                    "role": "user",
                    "content": f"""
ROLE:
Anda adalah asisten akademik yang bertugas mengolah dan menyederhanakan materi kuliah untuk mahasiswa tingkat sarjana.

TUJUAN UTAMA:
Dari teks yang diberikan, Anda harus memahami isinya secara menyeluruh, menemukan poin-poin inti, lalu menuliskannya ulang dalam bentuk penjelasan yang lebih mudah dipahami. Gunakan bahasa Indonesia yang jelas, intuitif, ringan, dan ramah mahasiswa, namun tetap akurat secara ilmiah. Anda juga diperbolehkan menggunakan analogi sederhana untuk membantu pemahaman—selama analogi tersebut tidak menambah atau mengubah substansi ilmiah dari teks.

ATURAN TAMBAHAN:
Anda tidak boleh menambahkan informasi baru. Jika ada bagian yang ambigu atau terpotong, gunakan apa adanya tanpa menebak. Jangan sertakan metadata, nomor halaman, atau hal yang tidak relevan. Hasil akhir harus berupa paragraf penjelasan yang runtut dan koheren. Setelah paragraf utama, tambahkan bullet points yang berisi ringkasan poin inti agar pembaca lebih mudah mencerna isinya. Bullet points hanya boleh merangkum informasi asli dari teks.

FORMAT OUTPUT:
1. Paragraf penjelasan yang mengalir dan terstruktur.
2. Bullet points berisi rangkuman inti.
3. Gunakan analogi sederhana yang relevan bila dapat membantu pemahaman.

TEKS YANG HARUS DIOLAH:
{chunks}
"""


                }
            ]
        }
    )

    return response.json()["choices"][0]["message"]["content"]


In [90]:
answer = summarize_chunk(chunks)



In [91]:
print(answer)

Dalam era di mana teknologi Generative AI (GenAI) seperti ChatGPT sudah jadi bagian tak terpisahkan dari tugas akademik, dosen di perguruan tinggi didorong untuk lebih menekankan proses belajar daripada sekadar hasil akhir. Bayangkan GenAI seperti asisten pintar yang bisa bantu cari ide atau ringkas materi, tapi dosen harus memotivasi mahasiswa melalui umpan balik evaluatif, reflektif, dan kesempatan revisi, sambil menerapkan metode seperti Student-Centered Learning (SCL), Higher Order Thinking (HOT), atau Flip Learning. Setiap pertemuan kelas bisa difokuskan pada diskusi presentasi tugas, di mana mahasiswa merefleksikan penggunaan GenAI—misalnya, bagaimana mereka buat prompt efektif, evaluasi akurasi output GenAI (termasuk bias dan kepercayaan), fact-check, edit ulang, serta bandingkan hasil mandiri dengan GenAI untuk latih metakognisi. Untuk cegah pelanggaran etika, dosen tekankan konsekuensi kecurangan dan beri panduan checklist seperti "Apakah gaya tulisan saya konsisten?" atau "Ap